In [1]:
import os

import numpy as np
import pandas as pd
import numpy.testing as npt
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

import sys
sys.path.append('..')

import modules.stats as st
import modules.plotting as pl
import modules.pandas_funcs as pf

In [2]:
% matplotlib auto

Using matplotlib backend: MacOSX


In [3]:
results_dir = os.path.join('..', 'data', 'results')
match_dir = os.path.join('..', 'data', 'matching')


df_k_raw = pd.read_csv(os.path.join(results_dir, 'kinect_gait_metrics.csv'),
                       index_col=0)

df_z_raw = pd.read_csv(os.path.join(results_dir, 'zeno_gait_metrics.csv'),
                       index_col=0)

df_match = pd.read_csv(os.path.join(match_dir, 'match_kinect_zeno.csv'))

# Drop rows where file has no match
df_match = df_match.dropna(axis=0)


df_match_zeno = pd.merge(df_match, df_z_raw, left_on='Zeno', right_index=True)

df_total = pd.merge(df_match_zeno, df_k_raw, left_on='Kinect',
                    right_index=True, suffixes=('_z', '_k'))


# Take columns from total DataFrame to get Kinect and Zeno data
df_k = df_total.filter(like='_k')
df_z = df_total.filter(like='_z')

# Remove suffixes from column names
df_k = df_k.rename(columns=lambda x: str(x)[:-2])
df_z = df_z.rename(columns=lambda x: str(x)[:-2])


In [4]:
df_error = abs((df_k - df_z) / df_z) * 100

In [5]:
df_error.median()

stance_percentage_L     8.757610
stance_percentage_R    10.621991
step_length_L           7.782810
step_length_R          12.410485
stride_length_L        11.113202
stride_length_R         9.943766
stride_time_L          19.519977
stride_time_R          16.783494
stride_velocity_L       7.234475
stride_velocity_R       7.152990
stride_width_L         39.025586
stride_width_R         39.428879
dtype: float64

In [6]:
np.median(df_error.values)

11.38293689740369

In [7]:
# abs_step_length_L = np.sqrt(df_z.step_length_L ** 2 + df_z.stride_width_L ** 2)
# abs(abs_step_length_L - df_z.absolute_step_length_L).max()

In [8]:
# abs_step_length_L = np.sqrt(df_k.step_length_L ** 2 + df_k.stride_width_L ** 2)
# abs(abs_step_length_L - df_k.absolute_step_length_L).idxmax()

In [9]:
max_median_index = df_error.median(axis=1).idxmax()

In [10]:
metric = 'stride_length_R'

In [11]:
plt.figure()
df_error[metric].plot.bar()
plt.show()

In [12]:
max_index = df_error[metric].idxmax()

In [13]:
df_compare = pd.concat([df_k.loc[max_median_index], df_z.loc[max_median_index]], axis=1, sort=True)
df_compare.columns = ['Kinect', 'Zeno']
df_compare

,Kinect,Zeno
stance_percentage_L,69.609280,58.346
stance_percentage_R,66.881868,56.535
step_length_L,60.861535,59.106
step_length_R,57.693614,59.000
stride_length_L,119.310198,121.857
stride_length_R,116.838915,120.005
stride_time_L,0.850000,1.282
stride_time_R,0.875000,1.309
stride_velocity_L,138.094118,106.227
stride_velocity_R,138.425196,105.762


In [14]:
df_match.loc[max_median_index]

Zeno      B_2014-12-22_P005_Pre_tw_C
Kinect       2014-12-22_P005_Pre_003
Name: 42, dtype: object